# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.data.datapath import DataPath
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azureml.core import Workspace, Dataset
from azureml.train.automl.utilities import get_primary_metrics
from azureml.widgets import RunDetails
from azureml.core.run import Run
from azureml.train.automl.run import AutoMLRun

from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice

from azureml.core.authentication import InteractiveLoginAuthentication
import requests
import json
from azureml.core import Workspace, Model, Webservice

import urllib.request
import json
import os
import ssl
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.49.0


Must download config file from Azure ML UI and put the config.json in the same directory

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-233364
aml-quickstarts-233364
westus2
3d1a56d2-7c81-4118-9790-f85d1acf0c77


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "Project-3-Compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 0, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
# choose a name for experiment
experiment_name = 'Project_3_Experiment_AutoML'
experiment=Experiment(ws, experiment_name)

### Upload Data

Must Upload the data manually before running this part

In [5]:
dataset = Dataset.get_by_name(ws, name='behavioral_data')

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

Target: GENHLTH - general health

In [6]:
get_primary_metrics('regression')

['spearman_correlation',
 'normalized_root_mean_squared_error',
 'r2_score',
 'normalized_mean_absolute_error']

In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_hours": 1,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'normalized_root_mean_squared_error'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "regression",
                             training_data=dataset,
                             label_column_name="GENHLTH",   
                             path = '.',
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Project_3_Experiment_AutoML,AutoML_4a5c5fed-840b-4b21-916c-9a592aae73e0,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Run this if your kernel is restarted

In [27]:
# remote_run = AutoMLRun(experiment, [run for run in experiment.get_runs()][0].id)

In [9]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
Project_3_Experiment_AutoML,AutoML_479441af-c3b3-442a-b0da-f81ce10b714b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [11]:
remote_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
#TODO: Save the best model

In [48]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-core, training version:1.50.0, current version:1.49.0
Package:azureml-dataprep, training version:4.10.6, current version:4.9.1
Package:azureml-dataprep-rslex, training version:2.17.5, current version:2.16.1
Package:azureml-dataset-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-defaults, training version:1.50.0, current version:1.49.0
Package:azureml-interpret, training version:1.50.0, current version:1.49.0
Package:azureml-mlflow, training version:1.50.0, current version:1.49.0
Package:azureml-pipeline-core, training version:1.50.0.post1, current version:1.49.0
Package:azureml-responsibleai, training version:1.50.0, current version:1.49.0
Package:azureml-telemetry, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-client, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-runtime, training version:1.50.0, current 

Run(Experiment: Project_3_Experiment_AutoML,
Id: AutoML_0fc27147-a804-4625-89c0-e1b55d186c58_20,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/t...
                                             PreFittedSoftVotingRegressor(estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmregressor', LightGBMRegressor(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('1', Pipeline(memory=None, steps=[('maxabsscaler', Ma

In [50]:
best_run.id

'AutoML_0fc27147-a804-4625-89c0-e1b55d186c58_20'

In [49]:
best_run.get_metrics()

{'normalized_root_mean_squared_error': 0.10082166967691142,
 'root_mean_squared_log_error': 0.23816938096473475,
 'explained_variance': 0.48130265187921506,
 'normalized_median_absolute_error': 0.0653096154903828,
 'normalized_root_mean_squared_log_error': 0.14798295673582676,
 'normalized_mean_absolute_error': 0.07784708716317647,
 'mean_absolute_percentage_error': 32.22403919573907,
 'mean_absolute_error': 0.6227766973054117,
 'r2_score': 0.4812548911744846,
 'spearman_correlation': 0.6693378895384264,
 'root_mean_squared_error': 0.8065733574152913,
 'median_absolute_error': 0.5224769239230624,
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_0fc27147-a804-4625-89c0-e1b55d186c58_20/residuals',
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_0fc27147-a804-4625-89c0-e1b55d186c58_20/predicted_true'}

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [34]:
model = best_run.register_model(
    model_name='AutoML_Behavioral_Model', 
    model_path='outputs/model.pkl', 
    description='AutoML Model for Behavioral Dataset')

In [35]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-232431', subscription_id='9b72f9e6-56c5-4c16-991b-19c652994860', resource_group='aml-quickstarts-232431'), name=AutoML_Behavioral_Model, id=AutoML_Behavioral_Model:1, version=1, tags={}, properties={})

In [36]:
best_run.download_files(output_directory='automl_output_dir')

In [37]:
env = Environment.from_conda_specification("AutoML-env", "automl_output_dir/outputs/conda_env_v_1_0_0.yml")
inference_config = InferenceConfig(entry_script='automl_output_dir/outputs/scoring_file_v_2_0_0.py',
                                   environment=env)

In [38]:
deployment_config = AciWebservice.deploy_configuration(
    enable_app_insights=True,
    auth_enabled=True) 

In [39]:
service = Model.deploy(ws, "behavioral-model-endpoint", [model], inference_config, deployment_config, overwrite=True)

In [40]:
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-05-01 20:35:33+00:00 Creating Container Registry if not exists.
2023-05-01 20:35:34+00:00 Registering the environment.
2023-05-01 20:35:35+00:00 Building image..
2023-05-01 20:55:53+00:00 Generating deployment configuration.
2023-05-01 20:55:54+00:00 Submitting deployment to compute..
2023-05-01 20:55:59+00:00 Checking the status of deployment behavioral-model-endpoint..
2023-05-01 20:58:05+00:00 Checking the status of inference endpoint behavioral-model-endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"


Create sample data from original file

In [41]:
pandas_df = dataset.to_pandas_dataframe()
pandas_df.pop('GENHLTH')
sample_pandas = pandas_df.iloc[:5]
df_dict = sample_pandas.to_dict('records')
data_dict = {"Inputs": {"data": df_dict}}

In [42]:
def allowSelfSignedHttps(allowed):
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context
allowSelfSignedHttps(True)


data =  data_dict
body = str.encode(json.dumps(data))

endpoints = Webservice.list(ws, model_name=model.name)
url = endpoints[0].scoring_uri

api_key = service.get_keys()[0]
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    print(response.read())
except urllib.error.HTTPError as error:
    print("Request failed with status code: " + str(error.code))

b'{"Results": [4.27616945127904, 2.560542597523808, 3.63977787743829, 4.806087360910714, 2.973141870722119]}'


In [ ]:
print("Scoring_uri: " + url)

TODO: In the cell below, print the logs of the web service and delete the service

In [43]:
service.get_logs()

'/bin/bash: /azureml-envs/azureml_399903ab218c6bc60c3ac2efb6ffbdcd/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_399903ab218c6bc60c3ac2efb6ffbdcd/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_399903ab218c6bc60c3ac2efb6ffbdcd/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_399903ab218c6bc60c3ac2efb6ffbdcd/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n2023-05-01T20:57:47,008161800+00:00 - iot-server/run \n2023-05-01T20:57:47,007852300+00:00 - rsyslog/run \nbash: /azureml-envs/azureml_399903ab218c6bc60c3ac2efb6ffbdcd/lib/libtinfo.so.6: no version information available (required by bash)\n2023-05-01T20:57:47,025982800+00:00 - gunicorn/run \n2023-05-01T20:57:47,031034800+00:00 | gunicorn/run | \n2023-05-01T20:57:47,035790100+00:00 - nginx/run \n2023-05-01T20:57:47,104078300

In [ ]:
service.delete()

# Cleanup

In [ ]:
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
